# Small Experiments Pertaining to Shaders for Scientific Visualisation
The contents of this notebook is neither documentation nor does it serve as permanent unit tests for the project. It's purely kept as a record of some of the experimentation done while working on this project.

### Custom HTML Output in Jupyter

In [2]:
class CustomHTMLObject:
    def __init__(self, name: str, colour: str):
        """
        
        :param name: The name of the object
        :param colour: The hex colour code of the object (eg: "#112233")
        """
        self.name = name
        self.colour = colour
        
    def _repr_html_(self) -> str:
        """
        Hooks into Jupyter notebook rich display system, which calls _repr_html_ by
        default if an object is returned at the end of a cell.
        """
        return f"""
        <div class="custom-html-object">
            <div style="color: {self.colour};font-size: large;">{self.name}</div>
        </div>"""
    
# Now return a CustomHTMLObject to the cell
CustomHTMLObject("Custom HTML is cool!", "#ff2010")

### Running OpenGL Headless

In [1]:
%pip install moderngl

     ---------------------------------------- 0.0/106.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/106.8 kB ? eta -:--:--
     -------------------------------------- 106.8/106.8 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for glcontext<3,>=2.3.6 from https://files.pythonhosted.org/packages/75/65/a4cd375ff65099f9b6203a960337c24d21d12162107b0fe4f55dd617d2e8/glcontext-2.4.0-cp310-cp310-win_amd64.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from PIL import Image
import moderngl

In [34]:
# This snippet for detecting the python environment is taken from: https://github.com/InsightSoftwareConsortium/itkwidgets/blob/main/itkwidgets/integrations/environment.py
# Licensed under the Apache License 2.0: https://github.com/InsightSoftwareConsortium/itkwidgets/blob/main/LICENSE
from enum import Enum
from importlib import import_module
import sys


class Env(Enum):
    JUPYTER_NOTEBOOK = 'notebook'
    JUPYTERLAB = 'lab'
    JUPYTERLITE = 'lite'
    SAGEMAKER = 'sagemaker'
    HYPHA = 'hypha'
    COLAB = 'colab'


def find_env():
    try:
        from google.colab import files
        return Env.COLAB
    except:
        try:
            from IPython import get_ipython
            parent_header = get_ipython().parent_header
            username = parent_header['header']['username']
            if username == '':
                return Env.JUPYTERLAB
            elif username == 'username':
                return Env.JUPYTER_NOTEBOOK
            else:
                return Env.SAGEMAKER
        except:
            import sys
            if sys.platform == 'emscripten':
                return Env.JUPYTERLITE
            return Env.HYPHA


ENVIRONMENT = find_env()

In [35]:
# Setup the OpenGL context
if ENVIRONMENT == Env.COLAB:
    # In Google Colab we need to explicitly specify the EGL backend, otherwise it tries (and fails) to use X11
    ctx = moderngl.create_context(standalone=True, backend="egl")
else:
    # Otherwise let ModernGL try to automatically determine the correct backend
    ctx = moderngl.create_context(standalone=True)
print(f"Got OpenGL context:\n\tGL_VENDOR={ctx.info['GL_VENDOR']}\n\tGL_RENDERER={ctx.info['GL_RENDERER']}\n\tGL_VERSION={ctx.info['GL_VERSION']}")
from pprint import pprint
pprint(ctx.info)

resolution = (min(1024, ctx.info["GL_MAX_VIEWPORT_DIMS"][0]), 
              min(1024, ctx.info["GL_MAX_VIEWPORT_DIMS"][1]))
fbo = ctx.simple_framebuffer(resolution, components=4)
fbo.use()

# Create a vertex buffer
vertices = np.array([
   # X      Y      R    G    B
    -1.0,  -1.0,   1.0, 0.0, 0.0,
     1.0,  -1.0,   0.0, 1.0, 0.0,
     0.0,   1.0,   0.0, 0.0, 1.0],
    dtype='f4',
)

prog = ctx.program(vertex_shader="""
#version 330
in vec2 in_vert;
in vec3 in_color;
out vec3 color;
out vec2 position;
void main() {
    gl_Position = vec4(in_vert, 0.0, 1.0);
    color = in_color;
    position = in_vert*0.5+0.5;
}
""",
    fragment_shader="""
#version 330
out vec4 fragColor;
in vec3 color;
in vec2 position;

uniform vec2 iResolution;

float amod(float x, float y)
{
    return x - y * floor(x/y);
}

vec4 mainImage(in vec2 fragCoord)
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    float coord = floor(fragCoord.x) + floor(fragCoord.y/10.);
    //vec3 col = vec3(amod(coord, 16.)>=8.?1.:0., amod(coord, 32.)>=16.?1.:0., amod(coord, 64.)>=32.?1.:0.);
    //col = amod(coord, 128.)>64.?(col*0.3333+.3333):col;

    vec3 col = vec3(amod(coord, 64.) >= 56. ? 1. : 0.,
                    amod(coord + 16., 64.) >= 56. ? 1. : 0.,
                    amod(coord + 32., 64.) >= 56. ? 1. : 0.);
    col += amod(coord + 48., 64.) >= 56. ? 1. : 0.;
    col = amod(coord, 128.) > 64. ? (col * 0.3333 + .3333) : col;

    // Output to screen
    return vec4(col,1.0);
}

void main() {
    fragColor = mainImage(position * iResolution) + vec4(color, 1.0)*0.01;
    //fragColor = vec4(color, 1.0);
}
""",
)

# Set uniforms
prog["iResolution"].value = resolution

# Assign buffers and render
vao = ctx.simple_vertex_array(prog, ctx.buffer(vertices), 'in_vert', 'in_color')
vao.render(mode=moderngl.TRIANGLES)

# Save output to png
print("Saving image...")
image = Image.frombytes('RGBA', resolution, fbo.read(components=4))
image = image.transpose(Image.FLIP_TOP_BOTTOM)
image.save('openglHeadlessExample.png', format='png')
print(f"Saved to 'openglHeadlessExample.png'!")

Got OpenGL context:
	GL_VENDOR=NVIDIA Corporation
	GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
	GL_VERSION=3.3.0 NVIDIA 536.23
{'GL_ALIASED_LINE_WIDTH_RANGE': (1.0, 10.0),
 'GL_CONTEXT_PROFILE_MASK': 1,
 'GL_DOUBLEBUFFER': True,
 'GL_MAJOR_VERSION': 3,
 'GL_MAX_3D_TEXTURE_SIZE': 16384,
 'GL_MAX_ARRAY_TEXTURE_LAYERS': 2048,
 'GL_MAX_CLIP_DISTANCES': 8,
 'GL_MAX_COLOR_ATTACHMENTS': 8,
 'GL_MAX_COLOR_TEXTURE_SAMPLES': 32,
 'GL_MAX_COMBINED_FRAGMENT_UNIFORM_COMPONENTS': 233472,
 'GL_MAX_COMBINED_GEOMETRY_UNIFORM_COMPONENTS': 231424,
 'GL_MAX_COMBINED_TEXTURE_IMAGE_UNITS': 192,
 'GL_MAX_COMBINED_UNIFORM_BLOCKS': 84,
 'GL_MAX_COMBINED_VERTEX_UNIFORM_COMPONENTS': 233472,
 'GL_MAX_CUBE_MAP_TEXTURE_SIZE': 32768,
 'GL_MAX_DEPTH_TEXTURE_SAMPLES': 32,
 'GL_MAX_DRAW_BUFFERS': 8,
 'GL_MAX_DUAL_SOURCE_DRAW_BUFFERS': 1,
 'GL_MAX_ELEMENTS_INDICES': 1048576,
 'GL_MAX_ELEMENTS_VERTICES': 1048576,
 'GL_MAX_FRAGMENT_INPUT_COMPONENTS': 128,
 'GL_MAX_FRAGMENT_UNIFORM_BLOCKS': 14,
 'GL_MAX_FRAGMENT_UNIFO

In [37]:
print(ctx.extensions)
print(ctx.viewport)
print(ctx.extra)

{'GL_ARB_texture_non_power_of_two', 'GL_NV_timeline_semaphore', 'GL_SGIS_texture_lod', 'GL_EXT_gpu_program_parameters', 'GL_EXT_draw_instanced', 'GL_ARB_texture_env_dot3', 'GL_EXT_shader_image_load_formatted', 'GL_ARB_transpose_matrix', 'GL_EXT_memory_object', 'GL_NV_fragment_shader_interlock', 'GL_ARB_texture_filter_minmax', 'GL_ARB_internalformat_query2', 'GL_NV_transform_feedback2', 'GL_NV_memory_attachment', 'GL_ARB_robust_buffer_access_behavior', 'GL_ARB_polygon_offset_clamp', 'GL_NV_depth_buffer_float', 'GL_ARB_draw_buffers_blend', 'GL_NV_blend_square', 'GL_EXT_fog_coord', 'GL_EXT_pixel_buffer_object', 'GL_ARB_copy_image', 'GL_ARB_seamless_cubemap_per_texture', 'GL_NVX_linked_gpu_multicast', 'GL_ARB_depth_clamp', 'GL_NV_point_sprite', 'GL_NV_half_float', 'GL_ARB_timer_query', 'GL_ARB_post_depth_coverage', 'GL_ARB_shader_precision', 'GL_ARB_shader_viewport_layer_array', 'GL_ARB_occlusion_query', 'GL_NV_shader_atomic_fp16_vector', 'GL_EXT_blend_func_separate', 'GL_EXT_texture_shado

Exciting! So it seems that headless OpenGL rendering does indeed work with Google Colab (even when using a Colab session without a GPU surprisingly). Another thing to keep in mind though is that different platforms will have different capabilities, and we'll have to work out how to either handle those as errors (trying to allocate too many OpenGL buffers/samplers) or provide fallbacks (eg: tiled rendering for cases where the resolution is too high (on my machine the limit is 32k\*32k but on Colab the limit is 16k\*16k (for framebuffers and textures)). 

Don't know if this will present a problem, but it could be useful to hint to the graphics driver which GPU we want to use in multi GPU systems (such as laptops). As it turns out this is actually not standardised at all, and quite difficult in OpenGL since we don't create the context with a specific adapter (unlike DirectX). On Windows when using an Nvidia GPU we can set the "SHIM_MCCOMPAT=0x800000001" environment variable to specify the dedicated GPU; I'm not sure this works on other platforms, it's not well documented. It seems to me that on Google Colab at the moment, it only uses the software renderer with OpenGL (the Mesa llvmpipe driver) even when a GPU environment is used. For the purposes of development and testing this is probably _fine_, but to actually be useful we should really try to get GPU support working. From what I've read, it should just work ([https://stackoverflow.com/a/60937358](https://stackoverflow.com/a/60937358)) so I'm not sure what's wrong...

# ModernGL Threading

In [11]:
#  Copyright (c) 2023 Thomas Mathieson.
#  Distributed under the terms of the MIT license.
import logging
import time

import moderngl
import numpy as np


class SSVRenderOpenGL:
    """
    A rendering backend for SSV based on OpenGL
    """

    def __init__(self, resolution):
        self.resolution = resolution
        self.__create_context()
        self.start_time = time.time()

    def __create_context(self):
        """
        Creates an OpenGL context and a framebuffer.
        """
        self.ctx = moderngl.create_context(standalone=True)

        # To use moderngl with threading we need call the garbage collector manually
        self.ctx.gc_mode = "context_gc"

        resolution = (min(self.resolution[0], self.ctx.info["GL_MAX_VIEWPORT_DIMS"][0]),
                      min(self.resolution[1], self.ctx.info["GL_MAX_VIEWPORT_DIMS"][1]))
        self.fbo = self.ctx.simple_framebuffer(resolution, components=4)
        self.fbo.use()

    def log_context_info(self, full=False):
        """
        Logs the OpenGL information to the console for debugging.
        :param full: whether to log *all* of the OpenGL context information (including extensions)
        """
        print(f"Got OpenGL context:\n"
            f"\tGL_VENDOR={self.ctx.info['GL_VENDOR']}\n"
            f"\tGL_RENDERER={self.ctx.info['GL_RENDERER']}\n"
            f"\tGL_VERSION={self.ctx.info['GL_VERSION']}")
        if full:
            from pprint import pformat
            info = pformat(self.ctx.info, indent=4)
            print(f"Full info: \n{info}")
            extensions = pformat(self.ctx.extensions, indent=4)
            print(f"GL Extensions: \n{extensions}")

    def register_shader(self):
        ...

    def dbg_render_test(self):
        # Create a vertex buffer
        vertices = np.array([
            # X      Y      R    G    B
            -1.0, -1.0, 1.0, 0.0, 0.0,
            1.0, -1.0, 0.0, 1.0, 0.0,
            0.0, 1.0, 0.0, 0.0, 1.0],
            dtype='f4',
        )

        prog = self.ctx.program(vertex_shader="""
        #version 330
        in vec2 in_vert;
        in vec3 in_color;
        out vec3 color;
        out vec2 position;
        void main() {
            gl_Position = vec4(in_vert, 0.0, 1.0);
            color = in_color;
            position = in_vert*0.5+0.5;
        }
        """,
                                fragment_shader="""
        #version 330
        out vec4 fragColor;
        in vec3 color;
        in vec2 position;

        uniform vec2 iResolution;
        uniform float iTime;

        float amod(float x, float y)
        {
            return x - y * floor(x/y);
        }

        vec4 mainImage(in vec2 fragCoord)
        {
            // Normalized pixel coordinates (from 0 to 1)
            vec2 uv = fragCoord/iResolution.xy;

            float coord = floor(fragCoord.x) + floor(fragCoord.y/10.) + floor(iTime);
            //vec3 col = vec3(amod(coord, 16.)>=8.?1.:0., amod(coord, 32.)>=16.?1.:0., amod(coord, 64.)>=32.?1.:0.);
            //col = amod(coord, 128.)>64.?(col*0.3333+.3333):col;

            vec3 col = vec3(amod(coord, 64.) >= 56. ? 1. : 0.,
                            amod(coord + 16., 64.) >= 56. ? 1. : 0.,
                            amod(coord + 32., 64.) >= 56. ? 1. : 0.);
            col += amod(coord + 48., 64.) >= 56. ? 1. : 0.;
            col = amod(coord, 128.) > 64. ? (col * 0.3333 + .3333) : col;

            // Output to screen
            return vec4(col,1.0);
        }

        void main() {
            fragColor = mainImage(position * iResolution) + vec4(color, 1.0)*0.01;
            //fragColor = vec4(color, 1.0);
        }
        """)

        # Set uniforms
        prog["iResolution"].value = self.resolution
        prog["iTime"].value = time.time() - self.start_time

        # Assign buffers and render
        vao = self.ctx.simple_vertex_array(prog, self.ctx.buffer(vertices), 'in_vert', 'in_color')
        vao.render(mode=moderngl.TRIANGLES)

    def get_frame(self):
        """
        Gets the current contents of the frame buffer as a byte array.
        :return: the contents of the frame buffer as a bytearray in the ``RGBA`` format.
        """
        return self.fbo.read(components=4)


In [12]:
import threading


renderer = SSVRenderOpenGL((640,480))

def render_loop(exit_flag):
    target_framerate = 60
    last_frame_time = time.time()
    timeout = 0
    delta_time = 1/target_framerate
    while not exit_flag.wait(timeout=timeout):
        renderer.dbg_render_test()

        current_time = time.time()
        delta_time = current_time - last_frame_time
        timeout = max(1/target_framerate - delta_time, 0)
        last_frame_time = current_time
        break

def run():
    exit_flag = threading.Event()
    thread = threading.Thread(target=render_loop, args=(exit_flag,))

    # render_loop(exit_flag)

    thread.start()
    # self.render_loop(exit_flag)
    
run()

Traceback (most recent call last):
  File "C:\Users\Thoma\.conda\envs\IndProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Thoma\AppData\Local\Temp\ipykernel_49268\2383715671.py", line 12, in render_loop
  File "C:\Users\Thoma\AppData\Local\Temp\ipykernel_49268\740362812.py", line 63, in dbg_render_test
  File "C:\Users\Thoma\.conda\envs\IndProject\lib\site-packages\moderngl\__init__.py", line 1939, in program
    res.mglo, res._members, res._subroutines, res._geom, res._glo = self.mglo.program(
_moderngl.Error: cannot create program
